In [1]:
import cv2
import numpy as np
import pickle, os, sqlite3, random

image_x, image_y = 50, 50

def get_hand_hist():
    with open("hist", "rb") as f:
        hist = pickle.load(f)
    return hist

def init_create_folder_database():
    if not os.path.exists("gestures"):
        os.mkdir("gestures")
    if not os.path.exists("gesture_db.db"):
        conn = sqlite3.connect("gesture_db.db")
        create_table_cmd = "CREATE TABLE gesture ( g_id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE, g_name TEXT NOT NULL )"
        conn.execute(create_table_cmd)
        conn.commit()

def create_folder(folder_name):
    if not os.path.exists(folder_name):
        os.mkdir(folder_name)

def store_in_db(g_id, g_name):
    conn = sqlite3.connect("gesture_db.db")
    cmd = "INSERT INTO gesture (g_id, g_name) VALUES (%s, \'%s\')" % (g_id, g_name)
    try:
        conn.execute(cmd)
    except sqlite3.IntegrityError:
        choice = input("g_id already exists. Want to change the record? (y/n): ")
        if choice.lower() == 'y':
            cmd = "UPDATE gesture SET g_name = \'%s\' WHERE g_id = %s" % (g_name, g_id)
            conn.execute(cmd)
        else:
            print("Doing nothing...")
            return
    conn.commit()

    
def store_images(g_id):
    total_pics = 500
    hist = get_hand_hist()
    cam = cv2.VideoCapture(0)
    if cam.read()[0]==False:
        cam = cv2.VideoCapture(0)
    x, y, w, h = 300, 100, 300, 300

    create_folder("gestures/"+str(g_id))
    pic_no = 0
    flag_start_capturing = False
    frames = 0

    while True:
        img = cam.read()[1]
        img = cv2.flip(img, 1)
        imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([imgHSV], [0, 1], hist, [0, 180, 0, 256], 1)
        disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(10,10))
        cv2.filter2D(dst,-1,disc,dst)
        blur = cv2.GaussianBlur(dst, (11,11), 0)
        blur = cv2.medianBlur(blur, 15)
        thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
        thresh = cv2.merge((thresh,thresh,thresh))
        thresh = cv2.cvtColor(thresh, cv2.COLOR_BGR2GRAY)

       # edges = cv2.Canny(img, 40, 80)
        
        thresh = thresh[y:y+h, x:x+w]
        tmp = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        contours = tmp[0] if len(tmp) == 2 else tmp[1]

        if len(contours) > 0:
            contour = max(contours, key = cv2.contourArea)
            if cv2.contourArea(contour) > 10000 and frames > 50:
                x1, y1, w1, h1 = cv2.boundingRect(contour)
                pic_no += 1
                save_img = thresh[y1:y1+h1, x1:x1+w1]
                if w1 > h1:
                    save_img = cv2.copyMakeBorder(save_img, int((w1-h1)/2) , int((w1-h1)/2) , 0, 0, cv2.BORDER_CONSTANT, (0, 0, 0))
                elif h1 > w1:
                    save_img = cv2.copyMakeBorder(save_img, 0, 0, int((h1-w1)/2) , int((h1-w1)/2) , cv2.BORDER_CONSTANT, (0, 0, 0))
                save_img = cv2.resize(save_img, (image_x, image_y))
                rand = random.randint(0, 10)
                if rand % 2 == 0:
                    save_img = cv2.flip(save_img, 1)
                cv2.putText(img, "Capturing...", (30, 60), cv2.FONT_HERSHEY_TRIPLEX, 2, (127, 255, 255))
                cv2.imwrite("gestures/"+str(g_id)+"/"+str(pic_no)+".jpg", save_img)

        cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(img, str(pic_no), (30, 400), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (127, 127, 255))
        cv2.imshow("Capturing gesture", img)
        cv2.imshow("thresh", thresh)
        keypress = cv2.waitKey(1)
        if keypress == ord('c'):
            if flag_start_capturing == False:
                flag_start_capturing = True
            else:
                flag_start_capturing = False
                frames = 0
        if flag_start_capturing == True:
            frames += 1
        if pic_no == total_pics:
            break

init_create_folder_database()
g_id = input("Enter gesture no.: ")
g_name = input("Enter gesture name/text: ")
store_in_db(g_id, g_name)
store_images(g_id)

Enter gesture no.: 5
Enter gesture name/text: e
g_id already exists. Want to change the record? (y/n): y


In [1]:
import cv2, os

def flip_images():
    gest_folder = "gestures"
    images_labels = []
    images = []
    labels = []
    for g_id in os.listdir(gest_folder):
        for i in range(1200):
            path = gest_folder+"/"+g_id+"/"+str(i+1)+".jpg"
            new_path = gest_folder+"/"+g_id+"/"+str(i+1+1200)+".jpg"
            print(path)
            img = cv2.imread(path, 0)
            img = cv2.flip(img, 1)
            cv2.imwrite(new_path, img)

flip_images()

gestures/1/1.jpg
gestures/1/2.jpg
gestures/1/3.jpg
gestures/1/4.jpg
gestures/1/5.jpg
gestures/1/6.jpg
gestures/1/7.jpg
gestures/1/8.jpg
gestures/1/9.jpg
gestures/1/10.jpg
gestures/1/11.jpg
gestures/1/12.jpg
gestures/1/13.jpg
gestures/1/14.jpg
gestures/1/15.jpg
gestures/1/16.jpg
gestures/1/17.jpg
gestures/1/18.jpg
gestures/1/19.jpg
gestures/1/20.jpg
gestures/1/21.jpg
gestures/1/22.jpg
gestures/1/23.jpg
gestures/1/24.jpg
gestures/1/25.jpg
gestures/1/26.jpg
gestures/1/27.jpg
gestures/1/28.jpg
gestures/1/29.jpg
gestures/1/30.jpg
gestures/1/31.jpg
gestures/1/32.jpg
gestures/1/33.jpg
gestures/1/34.jpg
gestures/1/35.jpg
gestures/1/36.jpg
gestures/1/37.jpg
gestures/1/38.jpg
gestures/1/39.jpg
gestures/1/40.jpg
gestures/1/41.jpg
gestures/1/42.jpg
gestures/1/43.jpg
gestures/1/44.jpg
gestures/1/45.jpg
gestures/1/46.jpg
gestures/1/47.jpg
gestures/1/48.jpg
gestures/1/49.jpg
gestures/1/50.jpg
gestures/1/51.jpg
gestures/1/52.jpg
gestures/1/53.jpg
gestures/1/54.jpg
gestures/1/55.jpg
gestures/1/56.jpg
g

gestures/1/446.jpg
gestures/1/447.jpg
gestures/1/448.jpg
gestures/1/449.jpg
gestures/1/450.jpg
gestures/1/451.jpg
gestures/1/452.jpg
gestures/1/453.jpg
gestures/1/454.jpg
gestures/1/455.jpg
gestures/1/456.jpg
gestures/1/457.jpg
gestures/1/458.jpg
gestures/1/459.jpg
gestures/1/460.jpg
gestures/1/461.jpg
gestures/1/462.jpg
gestures/1/463.jpg
gestures/1/464.jpg
gestures/1/465.jpg
gestures/1/466.jpg
gestures/1/467.jpg
gestures/1/468.jpg
gestures/1/469.jpg
gestures/1/470.jpg
gestures/1/471.jpg
gestures/1/472.jpg
gestures/1/473.jpg
gestures/1/474.jpg
gestures/1/475.jpg
gestures/1/476.jpg
gestures/1/477.jpg
gestures/1/478.jpg
gestures/1/479.jpg
gestures/1/480.jpg
gestures/1/481.jpg
gestures/1/482.jpg
gestures/1/483.jpg
gestures/1/484.jpg
gestures/1/485.jpg
gestures/1/486.jpg
gestures/1/487.jpg
gestures/1/488.jpg
gestures/1/489.jpg
gestures/1/490.jpg
gestures/1/491.jpg
gestures/1/492.jpg
gestures/1/493.jpg
gestures/1/494.jpg
gestures/1/495.jpg
gestures/1/496.jpg
gestures/1/497.jpg
gestures/1/4

gestures/1/884.jpg
gestures/1/885.jpg
gestures/1/886.jpg
gestures/1/887.jpg
gestures/1/888.jpg
gestures/1/889.jpg
gestures/1/890.jpg
gestures/1/891.jpg
gestures/1/892.jpg
gestures/1/893.jpg
gestures/1/894.jpg
gestures/1/895.jpg
gestures/1/896.jpg
gestures/1/897.jpg
gestures/1/898.jpg
gestures/1/899.jpg
gestures/1/900.jpg
gestures/1/901.jpg
gestures/1/902.jpg
gestures/1/903.jpg
gestures/1/904.jpg
gestures/1/905.jpg
gestures/1/906.jpg
gestures/1/907.jpg
gestures/1/908.jpg
gestures/1/909.jpg
gestures/1/910.jpg
gestures/1/911.jpg
gestures/1/912.jpg
gestures/1/913.jpg
gestures/1/914.jpg
gestures/1/915.jpg
gestures/1/916.jpg
gestures/1/917.jpg
gestures/1/918.jpg
gestures/1/919.jpg
gestures/1/920.jpg
gestures/1/921.jpg
gestures/1/922.jpg
gestures/1/923.jpg
gestures/1/924.jpg
gestures/1/925.jpg
gestures/1/926.jpg
gestures/1/927.jpg
gestures/1/928.jpg
gestures/1/929.jpg
gestures/1/930.jpg
gestures/1/931.jpg
gestures/1/932.jpg
gestures/1/933.jpg
gestures/1/934.jpg
gestures/1/935.jpg
gestures/1/9

gestures/2/112.jpg
gestures/2/113.jpg
gestures/2/114.jpg
gestures/2/115.jpg
gestures/2/116.jpg
gestures/2/117.jpg
gestures/2/118.jpg
gestures/2/119.jpg
gestures/2/120.jpg
gestures/2/121.jpg
gestures/2/122.jpg
gestures/2/123.jpg
gestures/2/124.jpg
gestures/2/125.jpg
gestures/2/126.jpg
gestures/2/127.jpg
gestures/2/128.jpg
gestures/2/129.jpg
gestures/2/130.jpg
gestures/2/131.jpg
gestures/2/132.jpg
gestures/2/133.jpg
gestures/2/134.jpg
gestures/2/135.jpg
gestures/2/136.jpg
gestures/2/137.jpg
gestures/2/138.jpg
gestures/2/139.jpg
gestures/2/140.jpg
gestures/2/141.jpg
gestures/2/142.jpg
gestures/2/143.jpg
gestures/2/144.jpg
gestures/2/145.jpg
gestures/2/146.jpg
gestures/2/147.jpg
gestures/2/148.jpg
gestures/2/149.jpg
gestures/2/150.jpg
gestures/2/151.jpg
gestures/2/152.jpg
gestures/2/153.jpg
gestures/2/154.jpg
gestures/2/155.jpg
gestures/2/156.jpg
gestures/2/157.jpg
gestures/2/158.jpg
gestures/2/159.jpg
gestures/2/160.jpg
gestures/2/161.jpg
gestures/2/162.jpg
gestures/2/163.jpg
gestures/2/1

gestures/2/546.jpg
gestures/2/547.jpg
gestures/2/548.jpg
gestures/2/549.jpg
gestures/2/550.jpg
gestures/2/551.jpg
gestures/2/552.jpg
gestures/2/553.jpg
gestures/2/554.jpg
gestures/2/555.jpg
gestures/2/556.jpg
gestures/2/557.jpg
gestures/2/558.jpg
gestures/2/559.jpg
gestures/2/560.jpg
gestures/2/561.jpg
gestures/2/562.jpg
gestures/2/563.jpg
gestures/2/564.jpg
gestures/2/565.jpg
gestures/2/566.jpg
gestures/2/567.jpg
gestures/2/568.jpg
gestures/2/569.jpg
gestures/2/570.jpg
gestures/2/571.jpg
gestures/2/572.jpg
gestures/2/573.jpg
gestures/2/574.jpg
gestures/2/575.jpg
gestures/2/576.jpg
gestures/2/577.jpg
gestures/2/578.jpg
gestures/2/579.jpg
gestures/2/580.jpg
gestures/2/581.jpg
gestures/2/582.jpg
gestures/2/583.jpg
gestures/2/584.jpg
gestures/2/585.jpg
gestures/2/586.jpg
gestures/2/587.jpg
gestures/2/588.jpg
gestures/2/589.jpg
gestures/2/590.jpg
gestures/2/591.jpg
gestures/2/592.jpg
gestures/2/593.jpg
gestures/2/594.jpg
gestures/2/595.jpg
gestures/2/596.jpg
gestures/2/597.jpg
gestures/2/5

gestures/2/987.jpg
gestures/2/988.jpg
gestures/2/989.jpg
gestures/2/990.jpg
gestures/2/991.jpg
gestures/2/992.jpg
gestures/2/993.jpg
gestures/2/994.jpg
gestures/2/995.jpg
gestures/2/996.jpg
gestures/2/997.jpg
gestures/2/998.jpg
gestures/2/999.jpg
gestures/2/1000.jpg
gestures/2/1001.jpg
gestures/2/1002.jpg
gestures/2/1003.jpg
gestures/2/1004.jpg
gestures/2/1005.jpg
gestures/2/1006.jpg
gestures/2/1007.jpg
gestures/2/1008.jpg
gestures/2/1009.jpg
gestures/2/1010.jpg
gestures/2/1011.jpg
gestures/2/1012.jpg
gestures/2/1013.jpg
gestures/2/1014.jpg
gestures/2/1015.jpg
gestures/2/1016.jpg
gestures/2/1017.jpg
gestures/2/1018.jpg
gestures/2/1019.jpg
gestures/2/1020.jpg
gestures/2/1021.jpg
gestures/2/1022.jpg
gestures/2/1023.jpg
gestures/2/1024.jpg
gestures/2/1025.jpg
gestures/2/1026.jpg
gestures/2/1027.jpg
gestures/2/1028.jpg
gestures/2/1029.jpg
gestures/2/1030.jpg
gestures/2/1031.jpg
gestures/2/1032.jpg
gestures/2/1033.jpg
gestures/2/1034.jpg
gestures/2/1035.jpg
gestures/2/1036.jpg
gestures/2/10

In [2]:
import cv2
from glob import glob
import numpy as np
import random
from sklearn.utils import shuffle
import pickle
import os

def pickle_images_labels():
    images_labels = []
    images = glob("gestures/*/*.jpg")
    images.sort()
    for image in images:
        print(image)
        label = image[image.find(os.sep)+1: image.rfind(os.sep)]
        img = cv2.imread(image, 0)
        images_labels.append((np.array(img, dtype=np.uint8), int(label)))
    return images_labels

images_labels = pickle_images_labels()
images_labels = shuffle(shuffle(shuffle(shuffle(images_labels))))
images, labels = zip(*images_labels)
print("Length of images_labels", len(images_labels))

train_images = images[:int(5/6*len(images))]
print("Length of train_images", len(train_images))
with open("train_images", "wb") as f:
    pickle.dump(train_images, f)
del train_images

train_labels = labels[:int(5/6*len(labels))]
print("Length of train_labels", len(train_labels))
with open("train_labels", "wb") as f:
    pickle.dump(train_labels, f)
del train_labels

test_images = images[int(5/6*len(images)):int(11/12*len(images))]
print("Length of test_images", len(test_images))
with open("test_images", "wb") as f:
    pickle.dump(test_images, f)
del test_images

test_labels = labels[int(5/6*len(labels)):int(11/12*len(images))]
print("Length of test_labels", len(test_labels))
with open("test_labels", "wb") as f:
    pickle.dump(test_labels, f)
del test_labels

val_images = images[int(11/12*len(images)):]
print("Length of test_images", len(val_images))
with open("val_images", "wb") as f:
    pickle.dump(val_images, f)
del val_images

val_labels = labels[int(11/12*len(labels)):]
print("Length of val_labels", len(val_labels))
with open("val_labels", "wb") as f:
    pickle.dump(val_labels, f)
del val_labels

gestures\1\1.jpg
gestures\1\10.jpg
gestures\1\100.jpg
gestures\1\1000.jpg
gestures\1\1001.jpg
gestures\1\1002.jpg
gestures\1\1003.jpg
gestures\1\1004.jpg
gestures\1\1005.jpg
gestures\1\1006.jpg
gestures\1\1007.jpg
gestures\1\1008.jpg
gestures\1\1009.jpg
gestures\1\101.jpg
gestures\1\1010.jpg
gestures\1\1011.jpg
gestures\1\1012.jpg
gestures\1\1013.jpg
gestures\1\1014.jpg
gestures\1\1015.jpg
gestures\1\1016.jpg
gestures\1\1017.jpg
gestures\1\1018.jpg
gestures\1\1019.jpg
gestures\1\102.jpg
gestures\1\1020.jpg
gestures\1\1021.jpg
gestures\1\1022.jpg
gestures\1\1023.jpg
gestures\1\1024.jpg
gestures\1\1025.jpg
gestures\1\1026.jpg
gestures\1\1027.jpg
gestures\1\1028.jpg
gestures\1\1029.jpg
gestures\1\103.jpg
gestures\1\1030.jpg
gestures\1\1031.jpg
gestures\1\1032.jpg
gestures\1\1033.jpg
gestures\1\1034.jpg
gestures\1\1035.jpg
gestures\1\1036.jpg
gestures\1\1037.jpg
gestures\1\1038.jpg
gestures\1\1039.jpg
gestures\1\104.jpg
gestures\1\1040.jpg
gestures\1\1041.jpg
gestures\1\1042.jpg
gestures\1

gestures\1\138.jpg
gestures\1\1380.jpg
gestures\1\1381.jpg
gestures\1\1382.jpg
gestures\1\1383.jpg
gestures\1\1384.jpg
gestures\1\1385.jpg
gestures\1\1386.jpg
gestures\1\1387.jpg
gestures\1\1388.jpg
gestures\1\1389.jpg
gestures\1\139.jpg
gestures\1\1390.jpg
gestures\1\1391.jpg
gestures\1\1392.jpg
gestures\1\1393.jpg
gestures\1\1394.jpg
gestures\1\1395.jpg
gestures\1\1396.jpg
gestures\1\1397.jpg
gestures\1\1398.jpg
gestures\1\1399.jpg
gestures\1\14.jpg
gestures\1\140.jpg
gestures\1\1400.jpg
gestures\1\1401.jpg
gestures\1\1402.jpg
gestures\1\1403.jpg
gestures\1\1404.jpg
gestures\1\1405.jpg
gestures\1\1406.jpg
gestures\1\1407.jpg
gestures\1\1408.jpg
gestures\1\1409.jpg
gestures\1\141.jpg
gestures\1\1410.jpg
gestures\1\1411.jpg
gestures\1\1412.jpg
gestures\1\1413.jpg
gestures\1\1414.jpg
gestures\1\1415.jpg
gestures\1\1416.jpg
gestures\1\1417.jpg
gestures\1\1418.jpg
gestures\1\1419.jpg
gestures\1\142.jpg
gestures\1\1420.jpg
gestures\1\1421.jpg
gestures\1\1422.jpg
gestures\1\1423.jpg
gesture

gestures\1\1754.jpg
gestures\1\1755.jpg
gestures\1\1756.jpg
gestures\1\1757.jpg
gestures\1\1758.jpg
gestures\1\1759.jpg
gestures\1\176.jpg
gestures\1\1760.jpg
gestures\1\1761.jpg
gestures\1\1762.jpg
gestures\1\1763.jpg
gestures\1\1764.jpg
gestures\1\1765.jpg
gestures\1\1766.jpg
gestures\1\1767.jpg
gestures\1\1768.jpg
gestures\1\1769.jpg
gestures\1\177.jpg
gestures\1\1770.jpg
gestures\1\1771.jpg
gestures\1\1772.jpg
gestures\1\1773.jpg
gestures\1\1774.jpg
gestures\1\1775.jpg
gestures\1\1776.jpg
gestures\1\1777.jpg
gestures\1\1778.jpg
gestures\1\1779.jpg
gestures\1\178.jpg
gestures\1\1780.jpg
gestures\1\1781.jpg
gestures\1\1782.jpg
gestures\1\1783.jpg
gestures\1\1784.jpg
gestures\1\1785.jpg
gestures\1\1786.jpg
gestures\1\1787.jpg
gestures\1\1788.jpg
gestures\1\1789.jpg
gestures\1\179.jpg
gestures\1\1790.jpg
gestures\1\1791.jpg
gestures\1\1792.jpg
gestures\1\1793.jpg
gestures\1\1794.jpg
gestures\1\1795.jpg
gestures\1\1796.jpg
gestures\1\1797.jpg
gestures\1\1798.jpg
gestures\1\1799.jpg
gest

gestures\1\2132.jpg
gestures\1\2133.jpg
gestures\1\2134.jpg
gestures\1\2135.jpg
gestures\1\2136.jpg
gestures\1\2137.jpg
gestures\1\2138.jpg
gestures\1\2139.jpg
gestures\1\214.jpg
gestures\1\2140.jpg
gestures\1\2141.jpg
gestures\1\2142.jpg
gestures\1\2143.jpg
gestures\1\2144.jpg
gestures\1\2145.jpg
gestures\1\2146.jpg
gestures\1\2147.jpg
gestures\1\2148.jpg
gestures\1\2149.jpg
gestures\1\215.jpg
gestures\1\2150.jpg
gestures\1\2151.jpg
gestures\1\2152.jpg
gestures\1\2153.jpg
gestures\1\2154.jpg
gestures\1\2155.jpg
gestures\1\2156.jpg
gestures\1\2157.jpg
gestures\1\2158.jpg
gestures\1\2159.jpg
gestures\1\216.jpg
gestures\1\2160.jpg
gestures\1\2161.jpg
gestures\1\2162.jpg
gestures\1\2163.jpg
gestures\1\2164.jpg
gestures\1\2165.jpg
gestures\1\2166.jpg
gestures\1\2167.jpg
gestures\1\2168.jpg
gestures\1\2169.jpg
gestures\1\217.jpg
gestures\1\2170.jpg
gestures\1\2171.jpg
gestures\1\2172.jpg
gestures\1\2173.jpg
gestures\1\2174.jpg
gestures\1\2175.jpg
gestures\1\2176.jpg
gestures\1\2177.jpg
gest

gestures\1\654.jpg
gestures\1\655.jpg
gestures\1\656.jpg
gestures\1\657.jpg
gestures\1\658.jpg
gestures\1\659.jpg
gestures\1\66.jpg
gestures\1\660.jpg
gestures\1\661.jpg
gestures\1\662.jpg
gestures\1\663.jpg
gestures\1\664.jpg
gestures\1\665.jpg
gestures\1\666.jpg
gestures\1\667.jpg
gestures\1\668.jpg
gestures\1\669.jpg
gestures\1\67.jpg
gestures\1\670.jpg
gestures\1\671.jpg
gestures\1\672.jpg
gestures\1\673.jpg
gestures\1\674.jpg
gestures\1\675.jpg
gestures\1\676.jpg
gestures\1\677.jpg
gestures\1\678.jpg
gestures\1\679.jpg
gestures\1\68.jpg
gestures\1\680.jpg
gestures\1\681.jpg
gestures\1\682.jpg
gestures\1\683.jpg
gestures\1\684.jpg
gestures\1\685.jpg
gestures\1\686.jpg
gestures\1\687.jpg
gestures\1\688.jpg
gestures\1\689.jpg
gestures\1\69.jpg
gestures\1\690.jpg
gestures\1\691.jpg
gestures\1\692.jpg
gestures\1\693.jpg
gestures\1\694.jpg
gestures\1\695.jpg
gestures\1\696.jpg
gestures\1\697.jpg
gestures\1\698.jpg
gestures\1\699.jpg
gestures\1\7.jpg
gestures\1\70.jpg
gestures\1\700.jpg


gestures\2\1204.jpg
gestures\2\1205.jpg
gestures\2\1206.jpg
gestures\2\1207.jpg
gestures\2\1208.jpg
gestures\2\1209.jpg
gestures\2\121.jpg
gestures\2\1210.jpg
gestures\2\1211.jpg
gestures\2\1212.jpg
gestures\2\1213.jpg
gestures\2\1214.jpg
gestures\2\1215.jpg
gestures\2\1216.jpg
gestures\2\1217.jpg
gestures\2\1218.jpg
gestures\2\1219.jpg
gestures\2\122.jpg
gestures\2\1220.jpg
gestures\2\1221.jpg
gestures\2\1222.jpg
gestures\2\1223.jpg
gestures\2\1224.jpg
gestures\2\1225.jpg
gestures\2\1226.jpg
gestures\2\1227.jpg
gestures\2\1228.jpg
gestures\2\1229.jpg
gestures\2\123.jpg
gestures\2\1230.jpg
gestures\2\1231.jpg
gestures\2\1232.jpg
gestures\2\1233.jpg
gestures\2\1234.jpg
gestures\2\1235.jpg
gestures\2\1236.jpg
gestures\2\1237.jpg
gestures\2\1238.jpg
gestures\2\1239.jpg
gestures\2\124.jpg
gestures\2\1240.jpg
gestures\2\1241.jpg
gestures\2\1242.jpg
gestures\2\1243.jpg
gestures\2\1244.jpg
gestures\2\1245.jpg
gestures\2\1246.jpg
gestures\2\1247.jpg
gestures\2\1248.jpg
gestures\2\1249.jpg
gest

gestures\2\159.jpg
gestures\2\1590.jpg
gestures\2\1591.jpg
gestures\2\1592.jpg
gestures\2\1593.jpg
gestures\2\1594.jpg
gestures\2\1595.jpg
gestures\2\1596.jpg
gestures\2\1597.jpg
gestures\2\1598.jpg
gestures\2\1599.jpg
gestures\2\16.jpg
gestures\2\160.jpg
gestures\2\1600.jpg
gestures\2\1601.jpg
gestures\2\1602.jpg
gestures\2\1603.jpg
gestures\2\1604.jpg
gestures\2\1605.jpg
gestures\2\1606.jpg
gestures\2\1607.jpg
gestures\2\1608.jpg
gestures\2\1609.jpg
gestures\2\161.jpg
gestures\2\1610.jpg
gestures\2\1611.jpg
gestures\2\1612.jpg
gestures\2\1613.jpg
gestures\2\1614.jpg
gestures\2\1615.jpg
gestures\2\1616.jpg
gestures\2\1617.jpg
gestures\2\1618.jpg
gestures\2\1619.jpg
gestures\2\162.jpg
gestures\2\1620.jpg
gestures\2\1621.jpg
gestures\2\1622.jpg
gestures\2\1623.jpg
gestures\2\1624.jpg
gestures\2\1625.jpg
gestures\2\1626.jpg
gestures\2\1627.jpg
gestures\2\1628.jpg
gestures\2\1629.jpg
gestures\2\163.jpg
gestures\2\1630.jpg
gestures\2\1631.jpg
gestures\2\1632.jpg
gestures\2\1633.jpg
gesture

gestures\2\1970.jpg
gestures\2\1971.jpg
gestures\2\1972.jpg
gestures\2\1973.jpg
gestures\2\1974.jpg
gestures\2\1975.jpg
gestures\2\1976.jpg
gestures\2\1977.jpg
gestures\2\1978.jpg
gestures\2\1979.jpg
gestures\2\198.jpg
gestures\2\1980.jpg
gestures\2\1981.jpg
gestures\2\1982.jpg
gestures\2\1983.jpg
gestures\2\1984.jpg
gestures\2\1985.jpg
gestures\2\1986.jpg
gestures\2\1987.jpg
gestures\2\1988.jpg
gestures\2\1989.jpg
gestures\2\199.jpg
gestures\2\1990.jpg
gestures\2\1991.jpg
gestures\2\1992.jpg
gestures\2\1993.jpg
gestures\2\1994.jpg
gestures\2\1995.jpg
gestures\2\1996.jpg
gestures\2\1997.jpg
gestures\2\1998.jpg
gestures\2\1999.jpg
gestures\2\2.jpg
gestures\2\20.jpg
gestures\2\200.jpg
gestures\2\2000.jpg
gestures\2\2001.jpg
gestures\2\2002.jpg
gestures\2\2003.jpg
gestures\2\2004.jpg
gestures\2\2005.jpg
gestures\2\2006.jpg
gestures\2\2007.jpg
gestures\2\2008.jpg
gestures\2\2009.jpg
gestures\2\201.jpg
gestures\2\2010.jpg
gestures\2\2011.jpg
gestures\2\2012.jpg
gestures\2\2013.jpg
gestures\

gestures\2\2352.jpg
gestures\2\2353.jpg
gestures\2\2354.jpg
gestures\2\2355.jpg
gestures\2\2356.jpg
gestures\2\2357.jpg
gestures\2\2358.jpg
gestures\2\2359.jpg
gestures\2\236.jpg
gestures\2\2360.jpg
gestures\2\2361.jpg
gestures\2\2362.jpg
gestures\2\2363.jpg
gestures\2\2364.jpg
gestures\2\2365.jpg
gestures\2\2366.jpg
gestures\2\2367.jpg
gestures\2\2368.jpg
gestures\2\2369.jpg
gestures\2\237.jpg
gestures\2\2370.jpg
gestures\2\2371.jpg
gestures\2\2372.jpg
gestures\2\2373.jpg
gestures\2\2374.jpg
gestures\2\2375.jpg
gestures\2\2376.jpg
gestures\2\2377.jpg
gestures\2\2378.jpg
gestures\2\2379.jpg
gestures\2\238.jpg
gestures\2\2380.jpg
gestures\2\2381.jpg
gestures\2\2382.jpg
gestures\2\2383.jpg
gestures\2\2384.jpg
gestures\2\2385.jpg
gestures\2\2386.jpg
gestures\2\2387.jpg
gestures\2\2388.jpg
gestures\2\2389.jpg
gestures\2\239.jpg
gestures\2\2390.jpg
gestures\2\2391.jpg
gestures\2\2392.jpg
gestures\2\2393.jpg
gestures\2\2394.jpg
gestures\2\2395.jpg
gestures\2\2396.jpg
gestures\2\2397.jpg
gest

gestures\2\763.jpg
gestures\2\764.jpg
gestures\2\765.jpg
gestures\2\766.jpg
gestures\2\767.jpg
gestures\2\768.jpg
gestures\2\769.jpg
gestures\2\77.jpg
gestures\2\770.jpg
gestures\2\771.jpg
gestures\2\772.jpg
gestures\2\773.jpg
gestures\2\774.jpg
gestures\2\775.jpg
gestures\2\776.jpg
gestures\2\777.jpg
gestures\2\778.jpg
gestures\2\779.jpg
gestures\2\78.jpg
gestures\2\780.jpg
gestures\2\781.jpg
gestures\2\782.jpg
gestures\2\783.jpg
gestures\2\784.jpg
gestures\2\785.jpg
gestures\2\786.jpg
gestures\2\787.jpg
gestures\2\788.jpg
gestures\2\789.jpg
gestures\2\79.jpg
gestures\2\790.jpg
gestures\2\791.jpg
gestures\2\792.jpg
gestures\2\793.jpg
gestures\2\794.jpg
gestures\2\795.jpg
gestures\2\796.jpg
gestures\2\797.jpg
gestures\2\798.jpg
gestures\2\799.jpg
gestures\2\8.jpg
gestures\2\80.jpg
gestures\2\800.jpg
gestures\2\801.jpg
gestures\2\802.jpg
gestures\2\803.jpg
gestures\2\804.jpg
gestures\2\805.jpg
gestures\2\806.jpg
gestures\2\807.jpg
gestures\2\808.jpg
gestures\2\809.jpg
gestures\2\81.jpg


In [4]:
import cv2, os, random
import numpy as np

def get_image_size():
    img = cv2.imread('gestures/1/100.jpg', 0)
    return img.shape

gestures = os.listdir('gestures/')
gestures.sort(key = int)
begin_index = 0
end_index = 5
image_x, image_y = get_image_size()

if len(gestures)%5 != 0:
    rows = int(len(gestures)/5)+1
else:
    rows = int(len(gestures)/5)

full_img = None
for i in range(rows):
    col_img = None
    for j in range(begin_index, end_index):
        img_path = "gestures/%s/%d.jpg" % (j, random.randint(1, 1200))
        img = cv2.imread(img_path, 0)
        if np.any(img == None):
            img = np.zeros((image_y, image_x), dtype = np.uint8)
        if np.any(col_img == None):
            col_img = img
        else:
            col_img = np.hstack((col_img, img))

    begin_index += 5
    end_index += 5
    if np.any(full_img == None):
        full_img = col_img
    else:
        full_img = np.vstack((full_img, col_img))


cv2.imshow("gestures", full_img)
cv2.imwrite('full_img.jpg', full_img)
cv2.waitKey(0)

-1

In [5]:
import cv2
import numpy as np
import pickle

def build_squares(img):
    x, y, w, h = 420, 140, 10, 10
    d = 10
    imgCrop = None
    crop = None
    for i in range(10):
        for j in range(5):
            if np.any(imgCrop == None):
                imgCrop = img[y:y+h, x:x+w]
            else:
                imgCrop = np.hstack((imgCrop, img[y:y+h, x:x+w]))
            cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 1)
            x+=w+d
        if np.any(crop == None):
            crop = imgCrop
        else:
            crop = np.vstack((crop, imgCrop)) 
        imgCrop = None
        x = 420
        y+=h+d
    return crop

def get_hand_hist():
    cam = cv2.VideoCapture(0)
    if cam.read()[0]==False:
        cam = cv2.VideoCapture(0)
    x, y, w, h = 300, 100, 300, 300
    flagPressedC, flagPressedS = False, False
    imgCrop = None
    while True:
        img = cam.read()[1]
        img = cv2.flip(img, 1)
        img = cv2.resize(img, (640, 480))
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

        keypress = cv2.waitKey(1)
        if keypress == ord('c'):
            hsvCrop = cv2.cvtColor(imgCrop, cv2.COLOR_BGR2HSV)
            flagPressedC = True
            hist = cv2.calcHist([hsvCrop], [0, 1], None, [180, 256], [0, 180, 0, 256])
            cv2.normalize(hist, hist, 0, 255, cv2.NORM_MINMAX)
        elif keypress == ord('s'):
            flagPressedS = True	
            break
        if flagPressedC:	
            dst = cv2.calcBackProject([hsv], [0, 1], hist, [0, 180, 0, 256], 1)
            dst1 = dst.copy()
            disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(10,10))
            cv2.filter2D(dst,-1,disc,dst)
            blur = cv2.GaussianBlur(dst, (11,11), 0)
            blur = cv2.medianBlur(blur, 15)
            ret,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            thresh = cv2.merge((thresh,thresh,thresh))
            lower_red = np.array([30, 150, 50])  
            upper_red = np.array([255, 255, 180])  
  
    # create a red HSV colour boundary and    
    # threshold HSV image   
            mask = cv2.inRange(thresh, lower_red, upper_red) 
            res = cv2.bitwise_and(img, img, mask=mask) 
            edges = cv2.Canny(img, 40, 80)
            cv2.imshow('Edges', edges)  
            # cv2.imshow("Thresh", thresh)
            
        if not flagPressedS:
            imgCrop = build_squares(img)
        cv2.imshow("Set hand histogram", img)
    cam.release()
    cv2.destroyAllWindows()
    with open("hist", "wb") as f:
        pickle.dump(hist, f)


get_hand_hist()

In [4]:
# import libraries of python OpenCV    
import cv2  
  
# import Numpy by alias name np  
import numpy as np  
# capture frames from a camera   
cap = cv2.VideoCapture(0)  
  
# loop runs if capturing has been initialized   
while (1):  
  
    # reads frames from a camera   
    ret, frame = cap.read()  
  
    # converting BGR to HSV   
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)  
    # define range of red color in HSV   
    lower_red = np.array([30, 150, 50])  
    upper_red = np.array([255, 255, 180])  
  
    # create a red HSV colour boundary and    
    # threshold HSV image   
    mask = cv2.inRange(hsv, lower_red, upper_red)  
  
    # Bitwise-AND mask and original image   
    res = cv2.bitwise_and(frame, frame, mask=mask)  
  
    # Display an original image   
    cv2.imshow('Original', frame)  
  
    # discovers edges in the input image image and   
    # marks them in the output map edges   
    edges = cv2.Canny(frame, 40, 80)  
  
    # Display edges in a frame   
    cv2.imshow('Edges', edges)  
  
    # Wait for Esc key to stop   
    k = cv2.waitKey(5) & 0xFF  
    if k == 27:  
        break  
  
# Close the window   
cap.release()  
  
# De-allocate any associated memory usage   
cv2.destroyAllWindows()  